In [1]:
%load_ext autoreload
%autoreload 2
    
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests10 as vuong_tests_fast
from vuong_test_base import *


In [2]:
class JointNormal1(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([params[0], 0.0], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())
    
    
class JointNormal2(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([0.0, params[0]], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())


def setup_shi(yn,xn):
    # model 1 grad, etc.
    nobs = yn.shape[0]
    model1_param = np.array([yn.mean()])
    model2_param = np.array([xn.mean()])
    
    model1_deriv = JointNormal1(yn,xn)
    ll1 = model1_deriv.loglikeobs(model1_param)
    grad1 =  model1_deriv.score_obs(model1_param).reshape( (nobs,1) )
    hess1 = model1_deriv.hessian(model1_param)
    
    
    model2_deriv = JointNormal2(yn,xn)
    ll2 = model2_deriv.loglikeobs(model2_param)
    grad2 =  model2_deriv.score_obs(model2_param).reshape( (nobs,1) )  
    hess2 = model2_deriv.hessian(model2_param)
    
    return ll1,grad1,hess1,model1_param,ll2,grad2,hess2,model2_param

def gen_data(beta= 1.5, nobs=250):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([beta,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs


def gen_data3(beta= 1.5, nobs=250):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([0,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs


yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
print(grad1.shape,hess1.shape)
#NOTE! Weird size distortions with shi's test when theta = .5....

(250, 1) (1, 1)


In [3]:
def _estimate_HV(grad, hess_total, ridge=1e-8):
    """
    grad: (n,p) per-observation score of the log-likelihood
    hess_total: (p,p) full-sample Hessian of the log-likelihood (sum over i)
    Returns:
    H_hat = -E[∂² log f]  (per-observation average curvature)
    V_hat = E[s s′]       (uncentered score second moment)
    """
    n = grad.shape[0]
    H_hat = -hess_total / n
    S = grad - grad.mean() # i think its supposed to be centered....
    V_hat = (S.T @ S) / n
    return H_hat, V_hat

def _trace_HinvV(H, V):
    return float(np.trace(np.linalg.solve(H, V)))


def diagnostics_grad_hess(grad, hess_total, name="", ridge=1e-8):
    n, p = grad.shape
    # Per-observation H and V
    H, V = estimate_HV_known_total(grad, hess_total, ridge=ridge)
    
    # Score mean (should be near 0 at MLE)
    m = grad.mean(axis=0)
    m_norm = float(np.linalg.norm(m))
    # Scale of score variance
    eigV = np.linalg.eigvalsh(V)
    eigH = np.linalg.eigvalsh(H)
    condH = np.linalg.cond(H)
    
    # Fisher-type check: V ≈ H if well-specified (rough check)
    fisher_gap = float(np.linalg.norm(V - H, ord='fro'))  # not zero under misspecification
    
    trHinvV = trace_HinvV(H, V)
    
    print(f"--- Diagnostics [{name}] ---")
    print(f"n={n}, p={p}")
    print(f"score mean norm ||E[s]|| ≈ {m_norm:.4g}  (should be small at MLE)")
    print(f"H eig min/max = {eigH.min():.6g}/{eigH.max():.6g}, cond(H)={condH:.3g}")
    print(f"V eig min/max = {eigV.min():.6g}/{eigV.max():.6g}")
    print(f"tr(H^-1 V) = {trHinvV:.6g}  (≈ p if well-specified; O(p) in general)")
    print(f"||V - H||_F = {fisher_gap:.6g}  (0 if well-specified; >0 under misspec.)")
    print("---------------------------")

def compute_optimal_epsilon(ll1, grad1, hess1, params1,
                 ll2, grad2, hess2, params2, alpha=0.05, ridge=1e-8,
                            min_epsilon=1e-6, max_epsilon=10.0):
    nobs = ll1.shape[0]
    idx_even = np.arange(0, nobs, 2)
    idx_odd  = np.arange(1, nobs, 2)

    # Variances
    sigma2_hat  = max(float(np.var(ll1 - ll2, ddof=0)),1e-2) #NOTE cheating a little bit with epsilon... keep it big enough
    sigmaA2_hat = float(np.var(ll1[idx_even], ddof=0)) if idx_even.size > 1 else 0.0
    sigmaB2_hat = float(np.var(ll2[idx_odd],  ddof=0)) if idx_odd.size  > 1 else 0.0
    sigma_hat   = np.sqrt(max(sigma2_hat, 1e-12))

    # Estimate (H, V) for each model
    H1_hat, V1_hat = _estimate_HV(grad1, hess1, ridge=ridge)
    H2_hat, V2_hat = _estimate_HV(grad2, hess2, ridge=ridge)
    try:
        tr1 = abs(_trace_HinvV(H1_hat, V1_hat))
        tr2 = abs(_trace_HinvV(H2_hat, V2_hat))
        tr_max = max(tr1, tr2)
    except np.linalg.LinAlgError:
        tr_max = 1.0

    # Paper constants
    z = norm.ppf(1 - alpha/2.0)
    phi_z = norm.pdf(z)
    delta_star_hat = (sigma_hat / 2.0) * (z - np.sqrt(4.0 + z**2))
    phi_arg = z - (delta_star_hat / max(sigma_hat, 1e-12))
    phi_term = norm.pdf(phi_arg)

    denom_PL = 4.0 * (sigma_hat**3 + 1e-12)
    CPL_num = delta_star_hat * (sigma_hat**2 - 2.0 * (sigmaA2_hat + sigmaB2_hat))
    C_PL_hat = phi_term * CPL_num / denom_PL

    denom_SD = np.sqrt(max((sigmaA2_hat + sigmaB2_hat)/2.0, 1e-12))
    C_SD_hat = 2.0 * phi_z * tr_max / denom_SD

    lnln_term = max(np.log(np.log(max(nobs, 3))), 1e-6)
    fallback = nobs**(-1/6) * (lnln_term**(1/3))

    valid = (np.isfinite(C_PL_hat) and np.isfinite(C_SD_hat) and
             C_PL_hat > 0 and C_SD_hat > 0)

    if valid:
        eps_hat = ((C_SD_hat/C_PL_hat)**(1/3)) * (nobs**(-1/6)) * (lnln_term**(1/3))
    else:
        eps_hat = fallback

    return float(np.clip(eps_hat, min_epsilon, max_epsilon))

def sw_test_stat(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, epsilon=.5,compute_v=True,print_stuff=False):
    nobs = ll1.shape[0]

    idx_even = np.arange(0, nobs, 2)         # 0-based indices: 0,2,4,...
    idx_odd  = np.arange(1, nobs, 2)         # 1,3,5,...

    # Regular loglikelihood ratio statistic
    llr = (ll1 - ll2).sum()
    # Split-sample difference statistic
    llr_split = ll1[idx_even].sum() - ll2[idx_odd].sum()
    # Regularized numerator (as in your expression)
    llr_reg = llr + epsilon * llr_split
    # Main variance
    omega2 = (ll1 - ll2).var()
    # Split-group variances
    omega_A2 = ll1[idx_even].var()
    omega_B2 = ll2[idx_odd].var()
    # Regularized variance
    omega_reg2 = (1 + epsilon) * omega2 + (epsilon**2 / 2) * (omega_A2 + omega_B2)
    omega_reg = np.sqrt(omega_reg2)

    if print_stuff:
        print('llr',llr,llr_split)
        print('omega',omega2,omega_A2,omega_B2,np.sqrt(omega2),np.sqrt(omega_reg2))
    if not compute_v:
        return llr_reg,omega_reg,nobs 
        
    V = compute_eigen2(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2) # If you want to try to bias-correct you can, this is optional
    return llr_reg,omega_reg,V,nobs 
    

def sw_test(ll1, grad1, hess1, params1,
            ll2, grad2, hess2, params2,
            epsilon=.5, alpha=0.05, biascorrect=False):

        
    llr_reg,omega_reg,V,nobs =  sw_test_stat(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, epsilon=epsilon)
    
    if biascorrect:
        llr_reg += V.sum() / 2

    test_stat = llr_reg / (omega_reg * np.sqrt(nobs))

    # Two-sided test
    reject_high = (test_stat >= norm.ppf(1 - alpha / 2))
    reject_low  = (test_stat <= norm.ppf(alpha / 2))
    return int(reject_high) + 2 * int(reject_low)

    
yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)


reg_idx = regular_test(ll1, grad1, hess1, params1,
            ll2, grad2, hess2, params2)


yn,xn,nobs = gen_data(nobs=250,beta=0)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
epsilon = compute_optimal_epsilon(ll1, grad1, hess1, params1,
                 ll2, grad2, hess2, params2, ridge=1e-8 , alpha=.05)
print(epsilon)
reg_idx = sw_test(ll1, grad1, hess1, params1,ll2, grad2, hess2, params2,epsilon =epsilon,   biascorrect=False, alpha=.05)
print(reg_idx)

yn,xn,nobs = gen_data3()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
epsilon = compute_optimal_epsilon(ll1, grad1, hess1, params1,
                 ll2, grad2, hess2, params2, ridge=1e-8 , alpha=.05)
print(epsilon)
reg_idx = sw_test(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2 ,epsilon =epsilon,   biascorrect=False, alpha=.05)
print(reg_idx)

0.2796880101783003
0
0.26894212632500625
2


In [4]:
def bootstrap_distr(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2,
                    epsilon=0.5, trials=500, seed=None, biascorrect=False):
    nobs = ll1.shape[0]
    #print('---------')
    llr_reg, omega_reg, V, nobs = sw_test_stat(
        ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, epsilon=epsilon,print_stuff=False
    )
    if biascorrect:
        llr_reg += V.sum() / 2
    test_stat = llr_reg / (omega_reg * np.sqrt(nobs))
    stat_dist = []

    test_stat0 = (ll1-ll2).sum()/np.sqrt( nobs*(ll1-ll2).var() )
    stats_s0 = []
    
    
    for i in range(trials):
        if seed is not None:
            np.random.seed(seed + i)
        sample = np.random.choice(np.arange(nobs), nobs, replace=True)
        ll1_s = ll1[sample]
        ll2_s = ll2[sample]

        # Don't need V for bootstrap stats (saves computation)
        llr_reg_s, omega_reg_s, nobs_in_s = sw_test_stat(
            ll1_s, grad1, hess1, params1, ll2_s, grad2, hess2, params2, epsilon=epsilon, compute_v=False,print_stuff=False
        )


        # bias correct V not directly available for bootstrap
        stat_s = (llr_reg_s - llr_reg) / (omega_reg_s * np.sqrt(nobs_in_s))
        stat_dist.append(stat_s)

        ###### real stat for comparison...
        llrs0 = ll1_s - ll2_s
        stats_s0.append(  llrs0.sum()/  np.sqrt(nobs*llrs0.var()) )
    if False:
        print('test_stat0', nobs_in_s, nobs, (np.array(stats_s0)-test_stat0).mean(),np.array(stats_s0).mean(),test_stat0 )
        print('test_stat', nobs_in_s, nobs, (np.array(stat_dist)-test_stat).mean(),np.array(stat_dist).mean(),test_stat )
    #print('----')
    return np.array(stat_dist), test_stat

def sw_bs_test_helper(stat_dist, stat_obs, alpha=.05, left=True, right=True, print_stuff=False):
    cv_upper = np.percentile(stat_dist, 100 * (1 - alpha / 2))
    cv_lower = np.percentile(stat_dist, 100 * (alpha / 2))
    if print_stuff:
        print(f"mean={stat_dist.mean():.3f}, cv_lower={cv_lower:.3f}, stat_obs={stat_obs:.3f}, cv_upper={cv_upper:.3f}")
    out = 0
    if right and (stat_obs > cv_upper):
        out = 1
    if left and (stat_obs < cv_lower):
        out += 2
    return out

def sw_bs_test(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2,
               alpha=.05, trials=500, epsilon=0.5, biascorrect=False, seed=None, print_stuff=False):
    stat_dist, test_stat = bootstrap_distr(
        ll1, grad1, hess1, params1, ll2, grad2, hess2, params2,
        epsilon=epsilon, trials=trials, seed=seed, biascorrect=biascorrect
    )
    return sw_bs_test_helper(stat_dist, test_stat, alpha=alpha, print_stuff=print_stuff)

# Example Usage:
# result = sw_bs_test(ll1, grad1, hess1, params1, ll2, grad2, hess2, params2, epsilon=0.5, trials=1000)

In [5]:
def monte_carlo2(total, gen_data, setup_shi, alpha=.05,  epsilon=.5,trials=500, biascorrect=False,epsilon_auto=True):
    # Count for [model1, tie, model2] for each test
    reg = np.zeros(3, dtype=int)
    sw  = np.zeros(3, dtype=int)
    swbs = np.zeros(3, dtype=int)

    # Tracking statistics
    llr_sum = 0.0
    llr_sq_sum = 0.0
    omega_sum = 0.0
    nobs_last = None

    for a in range(total):
        np.random.seed()
        yn, xn, nobs = gen_data()
        ll1, grad1, hess1, params1, ll2, grad2, hess2, params2 = setup_shi(yn, xn)
        if epsilon_auto:
            epsilon = compute_optimal_epsilon(ll1, grad1, hess1, params1,
                 ll2, grad2, hess2, params2, ridge=1e-8 , alpha=alpha)
        # Update LLR-related stats
        llrn = (ll1 - ll2).sum()
        omegan = np.sqrt((ll1 - ll2).var())
        llr_sum   += llrn
        llr_sq_sum += llrn ** 2
        omega_sum += omegan
        nobs_last = nobs

        # Run the tests
        reg_idx = regular_test(
            ll1, grad1, hess1, params1,
            ll2, grad2, hess2, params2,
            biascorrect=biascorrect, alpha=alpha
        )
        sw_idx = sw_test(
            ll1, grad1, hess1, params1,
            ll2, grad2, hess2, params2, epsilon = epsilon,
            alpha=alpha, biascorrect=biascorrect
        )
        swbs_idx = sw_bs_test(
            ll1, grad1, hess1, params1,
            ll2, grad2, hess2, params2, epsilon = epsilon,
            alpha=alpha, biascorrect=biascorrect, print_stuff=False
        )

        reg[reg_idx]   += 1
        sw[sw_idx]     += 1
        swbs[swbs_idx] += 1

    # Convert counts to rates/frequencies
    reg_freq = reg / total
    sw_freq = sw / total
    swbs_freq = swbs / total

    llr_mean = llr_sum / total
    llr_sd   = np.sqrt(llr_sq_sum / total - llr_mean ** 2)
    omega_scaled = omega_sum * np.sqrt(nobs_last) / total if nobs_last is not None else np.nan

    return reg_freq, sw_freq, swbs_freq, llr_mean, llr_sd, omega_scaled


In [6]:
nobs = 250
num_sims = 100
epsilon = .5
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn)

gen_data_ex = lambda : gen_data(nobs=nobs,beta=0)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,epsilon=epsilon,epsilon_auto=False)
print(mc_out)

(array([1., 0., 0.]), array([0.95, 0.02, 0.03]), array([0.74, 0.15, 0.11]), np.float64(10.689338555657264), np.float64(13.677819674398188), np.float64(19.26042859809282))


In [7]:

gen_data_ex = lambda : gen_data(nobs=nobs,beta=0)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,epsilon=epsilon,epsilon_auto=True)
print(mc_out)

(array([1., 0., 0.]), array([0.98, 0.02, 0.  ]), array([0.7 , 0.18, 0.12]), np.float64(11.143109705007605), np.float64(20.43285726640253), np.float64(17.65665739938718))


In [8]:

gen_data_ex = lambda : gen_data3(nobs=nobs,beta=.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,epsilon=epsilon)
print(mc_out)

(array([0.66, 0.  , 0.34]), array([0.94, 0.  , 0.06]), array([0.72, 0.07, 0.21]), np.float64(-18.903471101278846), np.float64(17.424700492168036), np.float64(22.050154635976128))


# size

In [9]:
gen_data_ex = lambda : gen_data(nobs=nobs,beta=.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,epsilon=epsilon)
print(mc_out)

(array([0.83, 0.  , 0.17]), array([0.96, 0.03, 0.01]), array([0.68, 0.2 , 0.12]), np.float64(6.191675890011836), np.float64(38.26157373552545), np.float64(39.81464107438891))


In [10]:
gen_data_ex = lambda : gen_data(nobs=1000,beta=.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,epsilon=epsilon)
print(mc_out)

(array([0.88, 0.  , 0.12]), array([0.94, 0.02, 0.04]), array([0.72, 0.12, 0.16]), np.float64(5.886063884163076), np.float64(81.94521174705753), np.float64(79.53846857845777))


In [11]:
gen_data_ex = lambda : gen_data(nobs=nobs,beta=1.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,epsilon=epsilon)
print(mc_out)

(array([0.87, 0.02, 0.11]), array([0.94, 0.03, 0.03]), array([0.66, 0.12, 0.22]), np.float64(5.62725250623223), np.float64(149.1460921783497), np.float64(117.50634345822448))


In [12]:
gen_data_ex = lambda : gen_data(nobs=500,beta=.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,epsilon=epsilon)
print(mc_out)

(array([0.89, 0.  , 0.11]), array([0.94, 0.  , 0.06]), array([0.7 , 0.12, 0.18]), np.float64(15.075616167620483), np.float64(60.68412808480761), np.float64(59.19440294904288))


# power 0.... need to see...

In [13]:
gen_data_ex = lambda : gen_data3(nobs=nobs,beta=1.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,epsilon=epsilon)
print(mc_out)

(array([0., 0., 1.]), array([0.1, 0. , 0.9]), array([0.22, 0.  , 0.78]), np.float64(-267.7615912839913), np.float64(33.12834155614398), np.float64(34.28938522057534))


In [14]:
gen_data_ex = lambda : gen_data3(nobs=500,beta=1.5)
mc_out = monte_carlo2(num_sims,gen_data_ex,setup_shi_ex,trials=500,epsilon=epsilon)
print(mc_out)

(array([0., 0., 1.]), array([0., 0., 1.]), array([0.04, 0.  , 0.96]), np.float64(-549.4799280151321), np.float64(39.172633841062556), np.float64(39.97946742179))
